# Combining pythong lightweigt components and container operations

This notebook demos: 

* Defining a Kubeflow pipeline with the KFP SDK, combinging python lightweight components operations and container operations
* Creating an experiment and submitting pipelines to the KFP run time environment using the KFP SDK 

Reference documentation: 
* https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/
* https://www.kubeflow.org/docs/pipelines/sdk/build-component/

### Prerequisites: Install or update the pipelines SDK

You may need to **restart your notebook kernel** after updating the KFP sdk.

This notebook is intended to be run from a Kubeflow notebook server. (From other environments, you would need to pass different arguments to the `kfp.Client` constructor.)

In [ ]:
# You may need to restart your notebook kernel after updating 
!python3 -m pip install kfp-server-api --upgrade --user
!python3 -m pip install kfp --upgrade --user

### Setup

In [ ]:
EXPERIMENT_NAME = 'Combining pythong lightweigt components and container operations'        # Name of the experiment in the UI
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'    # Base image used for components in the pipeline

In [ ]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components
from kfp import gcp


### Create pipeline component

#### Create a python function

In [ ]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b



#### Build a pipeline component from the function

In [ ]:
# Convert the function to a pipeline operation.
add_op = components.func_to_container_op(
    add,
    base_image=BASE_IMAGE, 
)

#### Create a container operations

In [ ]:
def echo_op(text1, text2):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "Text 1: $0"; echo "Text 2: $1"', text1, text2]
    )

### Build a pipeline using the component

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='MPip pipeline',
   description='Credit Modeling Pipeline'
)

def calc_pipeline(
   a:float =0,
   b:float =7,
):
    

    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)
    
    hello_world_task=echo_op(add_3_task.output,add_2_task.output)

### Compile and run the pipeline

Kubeflow Pipelines lets you group pipeline runs by *Experiments*. You can create a new experiment, or call `kfp.Client().list_experiments()` to see existing ones.
If you don't specify the experiment name, the `Default` experiment will be used.
​
You can directly run a pipeline given its function definition:

In [ ]:
# Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}
# Launch a pipeline run given the pipeline function definition
kfp.Client().create_run_from_pipeline_func(calc_pipeline, arguments=arguments, 
                                           experiment_name=EXPERIMENT_NAME)